## Downscale intrinsic for new stereo B extrinsics

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()

In [3]:
intrinsics_path = f"{root}/output/intrinsic_params.json"
with open(intrinsics_path, 'r') as f:
    intrinsic_params = json.load(f)

cam3_intrinsics = intrinsic_params['CAM_3']
cam4_intrinsics = intrinsic_params['CAM_4']

cam3_intrinsics

{'fx': 1649.4381508720908,
 'fy': 1643.2293130912683,
 'cx': 1861.996578308323,
 'cy': 1073.6248295269133,
 'D': [0.04765917124161265,
  -0.0811009010479358,
  -0.0023832192453621184,
  -0.009151200860058757,
  -0.21526908351698],
 'K': [[1649.4381508720908, 0.0, 1861.996578308323],
  [0.0, 1643.2293130912683, 1073.6248295269133],
  [0.0, 0.0, 1.0]],
 'reprojection_error': 0.387,
 'K_undistort': [[855.0935500588114, 0.0, 2814.0916859544327],
  [0.0, 1551.6408639848426, 1061.444793350544],
  [0.0, 0.0, 1.0]],
 'roi_undistort': [1849, 48, 1990, 2039],
 'count_bfore': 80,
 'count_after': 22}

## From 4K to 1080

In [8]:
def downscale_camera_intrinsics(calib, orig_res=(3840, 2160), new_res=(1920, 1080)):
    scale_x = new_res[0] / orig_res[0]
    scale_y = new_res[1] / orig_res[1]
    print(scale_x, scale_y)

    # Create a copy to modify (to avoid altering the input directly)
    new_calib = calib.copy()

    # Scale the principal camera parameters
    if "fx" in calib:
        new_calib["fx"] = calib["fx"] * scale_x
    if "fy" in calib:
        new_calib["fy"] = calib["fy"] * scale_y
    if "cx" in calib:
        new_calib["cx"] = calib["cx"] * scale_x
    if "cy" in calib:
        new_calib["cy"] = calib["cy"] * scale_y

    # Scale the intrinsic matrix K.
    if "K" in calib:
        K = calib["K"]
        # Assuming K is a 3x3 list-of-lists
        new_K = [
            [K[0][0] * scale_x, K[0][1], K[0][2] * scale_x],
            [K[1][0], K[1][1] * scale_y, K[1][2] * scale_y],
            [K[2][0], K[2][1], K[2][2]]
        ]
        new_calib["K"] = new_K

    # Scale the undistorted camera matrix K_undistort.
    if "K_undistort" in calib:
        K_ud = calib["K_undistort"]
        new_K_ud = [
            [K_ud[0][0] * scale_x, K_ud[0][1], K_ud[0][2] * scale_x],
            [K_ud[1][0], K_ud[1][1] * scale_y, K_ud[1][2] * scale_y],
            [K_ud[2][0], K_ud[2][1], K_ud[2][2]]
        ]
        new_calib["K_undistort"] = new_K_ud

    # Scale the ROI, if available. 
    # This code assumes that ROI is given either as [x, y, width, height] OR [x, y, x2, y2].
    if "roi_undistort" in calib:
        roi = calib["roi_undistort"]
        if len(roi) == 4:
            # Check if ROI is in [x, y, width, height] or [x, y, x2, y2].
            # We can use a heuristic: if (x + width) is less than new width, assume it's width.
            if roi[0] + roi[2] <= orig_res[0] and roi[1] + roi[3] <= orig_res[1]:
                # Format: [x, y, width, height]
                new_roi = [
                    roi[0] * scale_x,
                    roi[1] * scale_y,
                    roi[2] * scale_x,
                    roi[3] * scale_y,
                ]
            else:
                # Format: [x, y, x2, y2] (i.e. corner coordinates)
                new_roi = [
                    roi[0] * scale_x,
                    roi[1] * scale_y,
                    roi[2] * scale_x,
                    roi[3] * scale_y,
                ]
            new_calib["roi_undistort"] = new_roi

    # The distortion coefficients D are typically dimensionless, so no scaling is applied.
    # Other parameters (like reprojection_error, counts) are left unchanged.
    return new_calib

In [9]:
cam3_intrinsics

{'fx': 1649.4381508720908,
 'fy': 1643.2293130912683,
 'cx': 1861.996578308323,
 'cy': 1073.6248295269133,
 'D': [0.04765917124161265,
  -0.0811009010479358,
  -0.0023832192453621184,
  -0.009151200860058757,
  -0.21526908351698],
 'K': [[1649.4381508720908, 0.0, 1861.996578308323],
  [0.0, 1643.2293130912683, 1073.6248295269133],
  [0.0, 0.0, 1.0]],
 'reprojection_error': 0.387,
 'K_undistort': [[855.0935500588114, 0.0, 2814.0916859544327],
  [0.0, 1551.6408639848426, 1061.444793350544],
  [0.0, 0.0, 1.0]],
 'roi_undistort': [1849, 48, 1990, 2039],
 'count_bfore': 80,
 'count_after': 22}

In [13]:
new_cam3_intrinsics = downscale_camera_intrinsics(cam3_intrinsics, (3840, 2160), (1920, 1080))
new_cam3_intrinsics

0.5 0.5


{'fx': 824.7190754360454,
 'fy': 821.6146565456341,
 'cx': 930.9982891541615,
 'cy': 536.8124147634567,
 'D': [0.04765917124161265,
  -0.0811009010479358,
  -0.0023832192453621184,
  -0.009151200860058757,
  -0.21526908351698],
 'K': [[824.7190754360454, 0.0, 930.9982891541615],
  [0.0, 821.6146565456341, 536.8124147634567],
  [0.0, 0.0, 1.0]],
 'reprojection_error': 0.387,
 'K_undistort': [[427.5467750294057, 0.0, 1407.0458429772164],
  [0.0, 775.8204319924213, 530.722396675272],
  [0.0, 0.0, 1.0]],
 'roi_undistort': [924.5, 24.0, 995.0, 1019.5],
 'count_bfore': 80,
 'count_after': 22}

In [12]:
cam3_upscaled = downscale_camera_intrinsics(new_cam3_intrinsics, (1920, 1080), (3840, 2160))
cam3_upscaled

2.0 2.0


{'fx': 1649.4381508720908,
 'fy': 1643.2293130912683,
 'cx': 1861.996578308323,
 'cy': 1073.6248295269133,
 'D': [0.04765917124161265,
  -0.0811009010479358,
  -0.0023832192453621184,
  -0.009151200860058757,
  -0.21526908351698],
 'K': [[1649.4381508720908, 0.0, 1861.996578308323],
  [0.0, 1643.2293130912683, 1073.6248295269133],
  [0.0, 0.0, 1.0]],
 'reprojection_error': 0.387,
 'K_undistort': [[855.0935500588114, 0.0, 2814.0916859544327],
  [0.0, 1551.6408639848426, 1061.444793350544],
  [0.0, 0.0, 1.0]],
 'roi_undistort': [1849.0, 48.0, 1990.0, 2039.0],
 'count_bfore': 80,
 'count_after': 22}

In [14]:
cam3 = intrinsic_params['CAM_3']
cam4 = intrinsic_params['CAM_4']
cam3_1080_scale = downscale_camera_intrinsics(cam3, (3840, 2160), (1920, 1080))
cam4_1080_scale = downscale_camera_intrinsics(cam4, (3840, 2160), (1920, 1080))
intrinsic_params['CAM_3_1080'] = cam3_1080_scale
intrinsic_params['CAM_4_1080'] = cam4_1080_scale

# update the intrinsic_params.json file
with open(intrinsics_path, 'w') as f:
    json.dump(intrinsic_params, f)


0.5 0.5
0.5 0.5
